In [1]:
import os
base_path = os.path.dirname(os.path.abspath(""))
base_path = base_path + "/headers/"
print(base_path)
header_path = base_path + 'header_data_treatment.ipynb'
app_name = "add_thesis"
%run $header_path $app_name=app_name

C:\Users\raphael.wargon\Documents\GitHub\sorting_univ\script/headers/
add_thesis


# Load data

In [3]:
df_scanR = spark.read.format('parquet').load('file:\\' + scanR_path + '\\publications.parquet')
set_french_authors = spark.read.parquet('file:\\' + save_path + 'set_french_authors.parquet')

merge_path = databases_path + '\\merge\\'

merge_authors = spark.read.format('parquet').load('file:\\' + merge_path + 'matched_authors_2steps.parquet').withColumnRenamed('id','idref')

In [ ]:
set_french_authors.join(merge_authors.groupBy("author_id").agg(func.concat_ws(',', func.collect_set('idref')).alias('idref')), on ='author_id', how = 'inner').count()

In [4]:
merge_authors = merge_authors.withColumn('idref', func.regexp_replace(func.col('idref'), 'idref', ''))

# Create dissertations subset

In [5]:
theses_oa = (df_scanR
          .filter(func.col('type')=='thesis')
          .withColumn('authors', func.explode(func.col('authors')))
          .withColumn('idref', func.col('authors.person'))
          .withColumn('idref', func.regexp_replace(func.col('idref'), 'idref', ''))
         .select('idref','authors.role','id','authors.affiliations', 'authors.firstName', 'authors.lastName', 'authors.fullName', func.col('year').alias('thesis_year'))
         .join(merge_authors, on = 'idref', how= 'inner')
         )
theses_oa.cache()

DataFrame[idref: string, role: string, id: string, affiliations: array<struct<BNF:string,ISNI:string,acronym:array<string>,address:string,address-line:string,addresses:array<string>,alias_idref:array<string>,aliases:array<string>,city:string,comments:array<string>,country:string,datasource:string,department:array<string>,detected_countries:array<string>,docid:string,external_ids:array<struct<id_type:string,id_value:string>>,grid:array<string>,hal_docid:string,id:array<struct<asserted-by:string,id:string,id-type:string>>,idref:string,ids:array<struct<id:string,type:string>>,name:string,orgs:array<struct<name:string>>,place:array<string>,postal-code:string,rnsr:string,ror:string,structId:string,viaf:string,websites:array<string>,zipcode:string>>, firstName: string, lastName: string, fullName: string, thesis_year: bigint, author_id: string]

In [7]:
theses_oa.show()

+---------+--------------+---------------+--------------------+---------+----------------+--------------------+-----------+-----------+
|    idref|          role|             id|        affiliations|firstName|        lastName|            fullName|thesis_year|  author_id|
+---------+--------------+---------------+--------------------+---------+----------------+--------------------+-----------+-----------+
|225780895|        author|nnt2021reiml006|[{NULL, NULL, NUL...|  Annette|       Bächstädt|   Annette Bächstädt|       2021|A5080516570|
|055151221|directeurthese|nnt2021reiml006|                NULL|Christine|           Sukic|     Christine Sukic|       2021|A5019905891|
|077047133|directeurthese|nnt2021reiml006|                NULL|   Thomas|         Nicklas|      Thomas Nicklas|       2021|A5046114743|
|060260866| presidentjury|nnt2021reiml006|                NULL|  Stefano|           Simiz|       Stefano Simiz|       2021|A5071036035|
|055151221|    membrejury|nnt2021reiml006|      

In [22]:
#(theses_oa.groupBy('id')
# .agg(func.max( (func.col('role')=="directeurthese").cast('int')).alias('has_supervisor'),
#      func.max( (func.col('role')=="rapporteur").cast('int')).alias('has_ref'),
#      func.max( (func.col('role')=="membrejury").cast('int')).alias('has_jury'))
#.groupBy('has_supervisor','has_ref','has_jury').count()).show()

# Create the author-level dataset for theses

In [8]:
merged = (set_french_authors.join(theses_oa, how= 'left', on =['author_id']))
merged.cache()
merged.groupBy(func.col('id').isNotNull().cast('int').alias('has_link_these')).count().show()

+--------------+-------+
|has_link_these|  count|
+--------------+-------+
|             1|1144393|
|             0| 202888|
+--------------+-------+



In [9]:
thesis_supervisor = (merged
                     .withColumn('thesis_supervisor', func.concat_ws(',',func.array_compact(
                         func.collect_set( func.when(func.col('role')=="directeurthese", func.col('author_id')) ).over(Window.partitionBy('id')))))
                     .withColumn('thesis_jury', func.concat_ws(',',func.array_compact(
                         func.collect_set( func.when(func.col('role').contains("jury"), func.col('author_id')) ).over(Window.partitionBy('id')))))
                     .withColumn('thesis_referee', func.concat_ws(',',func.array_compact(
                         func.collect_set( func.when(func.col('role')=="rapporteur", func.col('author_id')) ).over(Window.partitionBy('id')))))
                     .filter(func.col('role')=='author')
                    )
thesis_supervisor.persist()
thesis_supervisor.show()

+-----------+---------+----------+-----------+---------+---------+------+--------------------+--------------------+-------------+-------------------+--------------------+-----------+-----------------+-----------+--------------+
|  author_id|last_year|entry_year|n_pub_in_FR|n_y_in_FR|    idref|  role|                  id|        affiliations|    firstName|           lastName|            fullName|thesis_year|thesis_supervisor|thesis_jury|thesis_referee|
+-----------+---------+----------+-----------+---------+---------+------+--------------------+--------------------+-------------+-------------------+--------------------+-----------+-----------------+-----------+--------------+
|A5053391421|     2020|      2014|         22|        7|235710385|author|doi10.1016/j.ispr...|[{NULL, NULL, NUL...|      Ewelina|             Rupnik|      Ewelina Rupnik|       2021|                 |           |              |
|A5102952297|     2020|      2002|          4|        4|114109478|author|doi10.1016/j.is

In [10]:
thesis_supervisor.groupBy( (func.col('thesis_supervisor')=="").cast('int').alias('has_supervisor'),
                           (func.col('thesis_jury')=="").cast('int').alias('has_jury'),
                          (func.col('thesis_referee')=="").cast('int').alias('has_referee')
                         ).count().show()

+--------------+--------+-----------+-----+
|has_supervisor|has_jury|has_referee|count|
+--------------+--------+-----------+-----+
|             0|       1|          1|61678|
|             0|       0|          0|35174|
|             1|       0|          1| 1873|
|             1|       1|          1|43561|
|             1|       0|          0| 4178|
|             1|       1|          0|  265|
|             0|       1|          0| 1053|
|             0|       0|          1| 8802|
+--------------+--------+-----------+-----+



In [11]:
panel_smoothed = (spark.read.parquet('file:\\' + save_path + 'panel_smoothed.parquet')
                    .withColumn('parent', func.regexp_replace(func.col('parent'), r'\}\,\{','};{'))
                    .withColumn('parent', func.regexp_replace(func.col('parent'), r'\[',''))
                    .withColumn('parent', func.regexp_replace(func.col('parent'), r'\]',''))
                    .withColumn('parent', func.when(func.col('parent').contains(';'), func.split(func.col('parent'), ';'))
                                .otherwise(func.array(func.col('parent'))))
                 .withColumn('parent', func.transform(func.col('parent'), lambda x: func.from_json(x, func.schema_of_json('{"parent_id":"","parent_name":"","parent_type":""}'))))
                 )

In [12]:
thesis_supervisor_af = (thesis_supervisor
                          .select('author_id', "thesis_year", func.explode(func.split(func.col('thesis_supervisor'), ",")).alias('thesis_supervisor'))
                        .join(panel_smoothed
                              .select(func.col('author_id').alias('thesis_supervisor'), func.col('year').alias('thesis_year'),
                                                    'inst_id', func.col('parent.parent_id').alias('parent_id')),
                              on = ['thesis_supervisor','thesis_year'], how = 'left'
                             )
                        .groupBy('author_id')
                        .agg(func.concat_ws(",", func.array_compact(func.collect_set('inst_id'))).alias('thesis_supervisor_inst'),
                             func.concat_ws(",", func.array_compact(func.flatten(func.collect_set('parent_id')))).alias('thesis_supervisor_parent_inst')
                             )
                       )

In [14]:
#thesis_supervisor_af.groupBy( (func.col('thesis_supervisor')!="").cast('int').alias('has_supervisor'), (func.col('thesis_supervisor_inst')!="").cast('int').alias('in_sample')).count().show()

In [15]:
thesis_ref_af = (thesis_supervisor
                          .select('author_id', "thesis_year", func.explode(func.split(func.col('thesis_referee'), ",")).alias('thesis_referee'))
                        .join(panel_smoothed
                              .select(func.col('author_id').alias('thesis_referee'), func.col('year').alias('thesis_year'),
                                                    'inst_id', func.col('parent.parent_id').alias('parent_id')),
                              on = ['thesis_referee','thesis_year'], how = 'left'
                             )
                        .groupBy('author_id')
                        .agg(func.concat_ws(",", func.array_compact(func.collect_set('inst_id'))).alias('thesis_referee_inst'),
                             func.concat_ws(",", func.array_compact(func.flatten(func.collect_set('parent_id')))).alias('thesis_referee_parent_inst')
                             )
                       )
thesis_jury_af = (thesis_supervisor
                          .select('author_id', "thesis_year", func.explode(func.split(func.col('thesis_jury'), ",")).alias('thesis_jury'))
                        .join(panel_smoothed
                              .select(func.col('author_id').alias('thesis_jury'), func.col('year').alias('thesis_year'),
                                                    'inst_id', func.col('parent.parent_id').alias('parent_id')),
                              on = ['thesis_jury','thesis_year'], how = 'left'
                             )
                        .groupBy('author_id')
                        .agg(func.concat_ws(",", func.array_compact(func.collect_set('inst_id'))).alias('thesis_jury_inst'),
                             func.concat_ws(",", func.array_compact(func.flatten(func.collect_set('parent_id')))).alias('thesis_jury_parent_inst')
                             )
                       )

In [16]:
thesis_info = (thesis_supervisor.select('author_id','thesis_year','thesis_supervisor', 'thesis_jury','thesis_referee')
               .join(thesis_supervisor_af, on = ['author_id'], how = 'left')
               .join(thesis_ref_af, on = ['author_id'], how = 'left')
               .join(thesis_jury_af, on = ['author_id'], how = 'left')
              )

In [17]:
thesis_info.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- thesis_year: long (nullable = true)
 |-- thesis_supervisor: string (nullable = false)
 |-- thesis_jury: string (nullable = false)
 |-- thesis_referee: string (nullable = false)
 |-- thesis_supervisor_inst: string (nullable = true)
 |-- thesis_supervisor_parent_inst: string (nullable = true)
 |-- thesis_referee_inst: string (nullable = true)
 |-- thesis_referee_parent_inst: string (nullable = true)
 |-- thesis_jury_inst: string (nullable = true)
 |-- thesis_jury_parent_inst: string (nullable = true)



In [18]:
thesis_info.write.mode('overwrite').parquet('file:\\' + save_path + 'thesis_info.parquet')

# Create the author-year level dataset for supervisors

In [19]:
supervised_students = (merged
                     .withColumn('supervised_student', func.concat_ws(',',func.array_compact(
                         func.collect_set( func.when(func.col('role')=="author", func.col('author_id')) ).over(Window.partitionBy('id')))))
                     .filter(func.col('role')=='directeurthese')
                       .withColumn('year', func.sequence(func.col('thesis_year')-3, func.col('thesis_year')))
                       .withColumn('year', func.explode(func.col('year')))
                       .groupBy('author_id','year')
                       .agg(func.concat_ws(',',func.array_compact(func.collect_set(func.col('supervised_student')))).alias('supervised_students'))
                       .withColumn('n_phd_students', func.when(func.col('supervised_students')!="", func.size(func.split(func.col('supervised_students'), ',')))
                                   .otherwise(func.lit(0)))
                      )
supervised_students.cache()
supervised_students.show()

+-----------+----+--------------------+--------------+
|  author_id|year| supervised_students|n_phd_students|
+-----------+----+--------------------+--------------+
|A5000167897|2020|                    |             0|
|A5000181960|2018|                    |             0|
|A5000220877|1999|                    |             0|
|A5000228910|2009|         A5080969673|             1|
|A5000235818|2004|        A5072133590,|             2|
|A5000240639|2010|         A5041543609|             1|
|A5000273598|1983|         A5078573429|             1|
|A5000347582|2012|        A5069214273,|             2|
|A5000414296|2016|                    |             0|
|A5000466174|2009|                    |             0|
|A5000467214|2007|A5048034351,,A505...|             3|
|A5000473093|2017|                    |             0|
|A5000474757|2014|         A5105707756|             1|
|A5000480458|1990|A5072071369,A5043...|             3|
|A5000530226|2022|         A5039185549|             1|
|A50005681

In [20]:
supervised_students.write.mode('overwrite').parquet('file:\\' + save_path + 'supervised_students.parquet')

# Add to  the panel

In [21]:
supervised_students= spark.read.parquet('file:\\' + save_path + 'supervised_students.parquet')

In [22]:
thesis_info = spark.read.parquet('file:\\' + save_path + 'thesis_info.parquet')

In [24]:
panel_smoothed

DataFrame[author_id: string, inst_id: string, year: int, new_inst_period: int, period_inst: bigint, last_y_inst_period: int, first_y_inst_period: int, n_inst_y: int, publications_raw: bigint, citations_raw: bigint, avg_rank_source_raw: double, nr_source_btm_50pct_raw: bigint, nr_source_mid_40pct_raw: bigint, nr_source_top_20pct_raw: bigint, nr_source_top_10pct_raw: bigint, nr_source_top_5pct_raw: bigint, inst_name: string, inst_type: string, country: string, parent: array<struct<parent_id:string,parent_name:string,parent_type:string>>, fused: int, uni_pub: int, cnrs: int, entry_year: int, last_year: int, all_y_in_FR: bigint, years_abroad: bigint, main_field: string, author_name: string, entry_cohort: double, publications: double, citations: double, avg_rank_source: double, nr_source_btm_50pct: double, nr_source_mid_40pct: double, nr_source_top_20pct: double, nr_source_top_10pct: double, nr_source_top_5pct: double]

In [25]:
panel_smoothed_w_theses = (panel_smoothed
                           .withColumn('inst_set_this_year', func.collect_set(func.col('inst_id')).over(Window.partitionBy('author_id','year')))
                           .join(thesis_info, on = ['author_id'], how = 'left')
                           .join(supervised_students, on = ['author_id','year'], how = 'left')
                           .withColumn('in_supervisor_inst', func.arrays_overlap(func.col('inst_set_this_year'), func.split(func.col('thesis_supervisor_inst'), ',')))
                           .withColumn('in_referee_inst', func.arrays_overlap(func.col('inst_set_this_year'), func.split(func.col('thesis_referee_inst'), ',')))
                           .withColumn('in_jury_inst', func.arrays_overlap(func.col('inst_set_this_year'),    func.split(func.col('thesis_jury_inst'), ',')))
                          )

In [26]:
panel_smoothed_w_theses.write.mode('overwrite').parquet('file:\\' + save_path + 'panel_smoothed_w_theses.parquet')

In [ ]:
panel_smoothed_w_theses= spark.read.parquet('file:\\' + save_path + 'panel_smoothed_w_theses.parquet')

In [78]:
spark.catalog.clearCache()

In [76]:
(panel_smoothed_w_theses
 .groupBy('year')
 .agg(*[func.mean(func.col(col).cast('int')).alias(col) for col in  ["in_supervisor_inst","in_referee_inst","in_jury_inst"]],
     func.mean('n_phd_students').alias('n_phd_students'))
.sort('year')).show(50)

+----+--------------------+--------------------+--------------------+------------------+
|year|  in_supervisor_inst|     in_referee_inst|        in_jury_inst|    n_phd_students|
+----+--------------------+--------------------+--------------------+------------------+
|1955|                 0.0|                 0.0|                 0.0|              NULL|
|1956|                 0.0|                 0.0|                 0.0|              NULL|
|1957|                 0.0|                 0.0|                 0.0|              NULL|
|1958|                 0.0|                 0.0|                 0.0|              NULL|
|1959|                 0.0|                 0.0|                 0.0|              NULL|
|1960|                 0.0|                 0.0|                 0.0|              NULL|
|1961|                 0.0|                 0.0|                 0.0|              NULL|
|1962|                 0.0|                 0.0|                 0.0|              NULL|
|1963|               

In [27]:
spark.stop()